# Summary

Link to paper: http://www.sciencedirect.com/science/article/pii/S0092867415004304

----

# Imports

In [1]:
NOTEBOOK_NAME = 'taipale_core'

In [2]:
%run imports.ipynb

protein_folding_energy
2016-05-09 15:46:16.307264


# Load mutation data

In [3]:
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

In [4]:
ls ../downloads/taipale/

1-s2.0-S0092867415004304-main.pdf  mmc2.xlsx  mmc4.xlsx  mmc6.xlsx  mmc8.pdf
mmc1.xlsx                          mmc3.xlsx  mmc5.xlsx  mmc7.xlsx


In [5]:
mutation_df = pd.read_excel('../downloads/taipale/mmc1.xlsx')

In [6]:
display(mutation_df.head(2))
print(mutation_df.shape[0])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,HGMD_accession,HGMD_variant_class,dbSNP_ID,Disease
0,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease
1,Disease mutation,A2M,2,2_18119,NM_000014:c.2998G>A,NP_000005:p.V1000I,CM980001,DP,rs669,"Alzheimer disease, association with"


2960


### mutation_df_1

In [7]:
mutation_df['refseq_id'] = mutation_df['Mutation_RefSeq_AA'].apply(lambda x: x.split(':')[0])
mutation_df['refseq_mutation'] = mutation_df['Mutation_RefSeq_AA'].apply(lambda x: x.split('.')[-1])

In [8]:
display(mutation_df.head(2))
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,HGMD_accession,HGMD_variant_class,dbSNP_ID,Disease,refseq_id,refseq_mutation
0,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease,NP_000005,C972Y
1,Disease mutation,A2M,2,2_18119,NM_000014:c.2998G>A,NP_000005:p.V1000I,CM980001,DP,rs669,"Alzheimer disease, association with",NP_000005,V1000I


Number of rows: 2960
Number of unique RefSeq nucleotide mutations: 2960
Number of unique RefSeq amino acid mutations: 2958


In [9]:
print("Duplicated RefSeq amino acid mutations:")
duplicates = mutation_df[mutation_df['Mutation_RefSeq_AA'].duplicated()]['Mutation_RefSeq_AA']
mutation_df[mutation_df['Mutation_RefSeq_AA'].isin(duplicates)]

Duplicated RefSeq amino acid mutations:


,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,HGMD_accession,HGMD_variant_class,dbSNP_ID,Disease,refseq_id,refseq_mutation
1404,Disease mutation,HSPB8,26353,26353_7864,NM_014365:c.423G>C,NP_055180:p.K141N,CM041377,DM,rs104894345,"Neuropathy, distal hereditary motor, type II",NP_055180,K141N
1405,Disease mutation,HSPB8,26353,26353_7865,NM_014365:c.423G>T,NP_055180:p.K141N,CM050270,DM,rs104894345,Charcot-Marie-Tooth disease 2L,NP_055180,K141N
1875,Disease mutation,NIPA1,123606,123606_15185,NM_144599:c.316G>A,NP_653200:p.G106R,CM050748,DM,rs104894490,"Spastic paraplegia, autosomal dominant",NP_653200,G106R
1876,Disease mutation,NIPA1,123606,123606_15186,NM_144599:c.316G>C,NP_653200:p.G106R,CM050749,DM,rs104894490,"Spastic paraplegia, autosomal dominant",NP_653200,G106R


In [10]:
mutation_df_1 = mutation_df.copy()

## NCBI to UniProt

Map NCBI to UniProt.

### mutation_df_2

Use the `uniprot_identifier` table to map Refseq to Uniprot


In [11]:
mutation_df = mutation_df_1.copy()

In [12]:
if os.path.isfile(op.join(NOTEBOOK_NAME, 'refseq_to_uniprot_2.pickle')):
    refseq_to_uniprot = pd.read_pickle(op.join(NOTEBOOK_NAME, 'refseq_to_uniprot_2.pickle'))
else:
    engine = sa.create_engine('mysql://strokach:@192.168.6.19:3306/uniprot_kb')
    sql_query = """\
    select *
    from uniprot_kb.uniprot_identifier
    join uniprot_kb.uniprot_sequence using (uniprot_id)
    where identifier_type = 'RefSeq'
    and (identifier_id like '{}.%%');
    """.format(".%%' or identifier_id like '".join(set(mutation_df['refseq_id'].values)))
    refseq_to_uniprot = pd.read_sql_query(sql_query, engine)
    
    os.makedirs(NOTEBOOK_NAME, exist_ok=True)
    refseq_to_uniprot.to_pickle(op.join(NOTEBOOK_NAME, 'refseq_to_uniprot_2.pickle'))

In [13]:
refseq_to_uniprot['refseq_id'] = refseq_to_uniprot['identifier_id'].apply(lambda x: x.split('.')[0])

In [14]:
print("DF mapping Refseq to Uniprot:")
display(refseq_to_uniprot.head(2))
print("Number of rows:", refseq_to_uniprot.shape[0])

DF mapping Refseq to Uniprot:


,uniprot_id,id,identifier_id,identifier_type,db,uniprot_name,protein_name,organism_name,gene_name,protein_existence,sequence_version,uniprot_sequence,refseq_id
0,P01023,61589,NP_000005.2,RefSeq,sp,A2MG_HUMAN,Alpha-2-macroglobulin,Homo sapiens,A2M,1,3,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,NP_000005
1,A4Z6T7,58633781,NP_000006.2,RefSeq,tr,A4Z6T7_HUMAN,Arylamine N-acetyltransferase 2,Homo sapiens,NAT2,3,1,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...,NP_000006


Number of rows: 1493


In [15]:
mutation_df = (
    mutation_df.merge(refseq_to_uniprot, on=['refseq_id'])
)

In [16]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 3902
Number of unique RefSeq nucleotide mutations: 2948
Number of unique RefSeq amino acid mutations: 2946 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 3902
Number of unique RefSeq / Uniprot amino acid mutations: 3900


Lost **12** amino acid mutations because RefSeq does not have a corresponding Uniprot ID

In [17]:
mutation_df_2 = mutation_df.copy()

## Mutation matches sequence

Make sure that the AA in UniProt sequence matches the AA in the RefSeq mutation.

### mutation_df_3

In [18]:
mutation_df = mutation_df_2.copy()

In [19]:
mutation_df['mutation_matches_sequence'] = (
    mutation_df[['refseq_mutation', 'uniprot_sequence']]
    .apply(lambda x: ascommon.sequence_tools.mutation_matches_sequence(*x), axis=1)
)
mutation_df = mutation_df[mutation_df['mutation_matches_sequence']]

In [20]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 3521
Number of unique RefSeq nucleotide mutations: 2875
Number of unique RefSeq amino acid mutations: 2873 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 3521
Number of unique RefSeq / Uniprot amino acid mutations: 3519


Lost **~70** more amino acid mutations because the RefSeq mutation did not match the Uniprot sequence

In [21]:
mutation_df_3 = mutation_df.copy()

## Protein has domain(s)

Select only those mutations that fall inside a protein domain.

### mutation_df_4

In [22]:
mutation_df = mutation_df_3.copy()

In [23]:
engine = sa.create_engine('mysql://strokach:@192.168.6.19:3306/elaspic')
sql_query = """\
select uniprot_domain_id, uniprot_id, model_domain_def
from elaspic.uniprot_domain
join elaspic.uniprot_domain_model using (uniprot_domain_id)
where uniprot_id in ('{}');
""".format("', '".join(set(mutation_df['uniprot_id'].values)))
uniprot_domain_model = pd.read_sql_query(sql_query, engine)

In [24]:
mutation_df = (
    mutation_df
    .merge(uniprot_domain_model, on=['uniprot_id'])
)

In [25]:
mutation_df['mutation_in_domain'] = (
    mutation_df[['refseq_mutation', 'model_domain_def']]
        .apply(lambda x: ascommon.sequence_tools.mutation_in_domain(*x), axis=1)
)

In [26]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 4865
Number of unique RefSeq nucleotide mutations: 2517
Number of unique RefSeq amino acid mutations: 2516 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 3096
Number of unique RefSeq / Uniprot amino acid mutations: 3095


More than **~300** additional mutations don't fall inside a protein with *any* structural domains.

In [27]:
mutation_df_4 = mutation_df.copy()

## Mutation in domain

### mutation_df_5

In [29]:
mutation_df = mutation_df_4.copy()

In [30]:
mutation_df = mutation_df[mutation_df['mutation_in_domain']]

In [31]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 2377
Number of unique RefSeq nucleotide mutations: 1942
Number of unique RefSeq amino acid mutations: 1941 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 2377
Number of unique RefSeq / Uniprot amino acid mutations: 2376


Another **~550** mutations don't fall inside a domain for which we have a structural model (mostly because there is no structural domain in that region, but also because we didn't get around to making that homology model yet).

In [32]:
mutation_df_5 =  mutation_df.copy()

# Load histone interaction data

### mmc2_s2a

In [33]:
mmc2_s2a = pd.read_excel('../downloads/taipale/mmc2.xlsx', 'Table S2A')

In [34]:
display(mmc2_s2a.head(2))
print(mmc2_s2a.shape[0])
print(mmc2_s2a.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])

assert not set(mmc2_s2a['Mutation_RefSeq_NT']) - set(mutation_df_1['Mutation_RefSeq_NT'])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,WT_ELISA_score,Mut_ELISA_score,Both_expressed,WT_interaction_score,Mut_interaction_score,Differential_Z_score,Interact,Quality_control_factor_common,Quality_control_factor_official
0,Disease mutation,A4GALT,53947,53947_1850,NM_017436:c.287G>A,NP_059132:p.C96Y,0.51,0.79,1,1.398952,4.766989,3.105452,yes,BAG2,BAG2
1,Disease mutation,A4GALT,53947,53947_1852,NM_017436:c.299C>T,NP_059132:p.S100L,0.51,0.71,1,1.398952,5.029331,3.347341,yes,BAG2,BAG2


16464
2402


In [35]:
mmc2_s2a.drop(pd.Index(['Category', 'Symbol', 'Entrez_Gene_ID', 'Allele_ID', 'Mutation_RefSeq_AA']), axis=1, inplace=True)

# Load FoldX predictions

### mmc2_s2b

In [36]:
mmc2_s2b = pd.read_excel('../downloads/taipale/mmc2.xlsx', 'Table S2B')

In [37]:
display(mmc2_s2b.head(2))

assert not set(mmc2_s2b['Mutation_RefSeq_NT']) - set(mutation_df_1['Mutation_RefSeq_NT'])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,FoldX_value
0,Disease mutation,ACAT1,38,38_16884,NM_000019:c.218A>C,NP_000010:p.Q73P,7.841320
1,Disease mutation,ACAT1,38,38_16885,NM_000019:c.278A>G,NP_000010:p.N93S,0.236651


In [38]:
mmc2_s2b.drop(pd.Index(['Category', 'Symbol', 'Entrez_Gene_ID', 'Allele_ID', 'Mutation_RefSeq_AA']), axis=1, inplace=True)

# Save

In [39]:
db = datapkg.DataFrameToMySQL(CONNECTION_STR, NOTEBOOK_NAME, STG_SERVER_IP, echo=False)

### mutation_df_5

In [40]:
mutation_df = mutation_df_5.copy()

In [41]:
assert mutation_df['mutation_matches_sequence'].all()
assert mutation_df['mutation_in_domain'].all()

In [42]:
mutation_df[['uniprot_id', 'refseq_mutation']].head()

,uniprot_id,refseq_mutation
2,P01023,C972Y
6,Q9NRG9,S263P
7,Q9NRG9,L381R
8,Q9NRG9,L430F
9,F1T0I5,A129T


In [43]:
mutation_df.columns = [datapkg.format_column(c) for c in mutation_df.columns]

In [44]:
display(mutation_df.head(2))
print(mutation_df.shape[0])

,category,symbol,entrez_gene_id,allele_id,mutation_refseq_nt,mutation_refseq_aa,hgmd_accession,hgmd_variant_class,dbsnp_id,disease,refseq_id,refseq_mutation,uniprot_id,id,identifier_id,identifier_type,db,uniprot_name,protein_name,organism_name,gene_name,protein_existence,sequence_version,uniprot_sequence,mutation_matches_sequence,uniprot_domain_id,model_domain_def,mutation_in_domain
2,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease,NP_000005,C972Y,P01023,61589,NP_000005.2,RefSeq,sp,A2MG_HUMAN,Alpha-2-macroglobulin,Homo sapiens,A2M,1,3,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,True,52474282,958:1270,True
6,Disease mutation,AAAS,8086,8086_5619,NM_015665:c.787T>C,NP_056480:p.S263P,CM010150,DM,rs121918550,Triple-A syndrome,NP_056480,S263P,Q9NRG9,798718245,NP_056480.1,RefSeq,sp,AAAS_HUMAN,Aladin,Homo sapiens,AAAS,1,1,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...,True,108303701,130:487,True


2377


In [45]:
columns_to_keep = [
    'category', 'symbol', 'entrez_gene_id', 'allele_id', 
    'mutation_refseq_nt', 'mutation_refseq_aa', 'hgmd_accession', 'hgmd_variant_class', 'dbsnp_id', 'disease', 
    # RefSeq info (processed)
    'refseq_id', 'refseq_mutation', 
    # UniProt info
    'uniprot_id', 'uniprot_domain_id', 'model_domain_def'
]

In [46]:
mutation_df[columns_to_keep].head(2)

,category,symbol,entrez_gene_id,allele_id,mutation_refseq_nt,mutation_refseq_aa,hgmd_accession,hgmd_variant_class,dbsnp_id,disease,refseq_id,refseq_mutation,uniprot_id,uniprot_domain_id,model_domain_def
2,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease,NP_000005,C972Y,P01023,52474282,958:1270
6,Disease mutation,AAAS,8086,8086_5619,NM_015665:c.787T>C,NP_056480:p.S263P,CM010150,DM,rs121918550,Triple-A syndrome,NP_056480,S263P,Q9NRG9,108303701,130:487


In [47]:
print(mutation_df.shape[0])
print(mutation_df.drop_duplicates(subset=['mutation_refseq_nt']).shape[0])
print('RefSeq...')
print(mutation_df.drop_duplicates(subset=['refseq_id', 'refseq_mutation']).shape[0])
print(mutation_df.drop_duplicates(subset=['refseq_id', 'mutation_refseq_nt']).shape[0])
print('Uniprot...')
print(mutation_df.drop_duplicates(subset=['uniprot_id', 'refseq_mutation']).shape[0])
print(mutation_df.drop_duplicates(subset=['uniprot_id', 'mutation_refseq_nt']).shape[0])

2377
1942
RefSeq...
1941
1942
Uniprot...
2376
2377


In [48]:
db.import_table(
    mutation_df[columns_to_keep], 
    'taipale_core', [
        [('refseq_id', 'refseq_mutation'), False],
        [('uniprot_id', 'refseq_mutation'), False],
        [('uniprot_id', 'mutation_refseq_nt'), True],
        [('uniprot_domain_id', 'refseq_mutation'), False],
    ],
)

FML
FML
FML
FML


### mmc2_s2a

In [49]:
df = mmc2_s2a.copy()

In [50]:
df.columns = [datapkg.format_column(c) for c in df.columns]

In [51]:
df.head()

,mutation_refseq_nt,wt_elisa_score,mut_elisa_score,both_expressed,wt_interaction_score,mut_interaction_score,differential_z_score,interact,quality_control_factor_common,quality_control_factor_official
0,NM_017436:c.287G>A,0.51,0.79,1,1.398952,4.766989,3.105452,yes,BAG2,BAG2
1,NM_017436:c.299C>T,0.51,0.71,1,1.398952,5.029331,3.347341,yes,BAG2,BAG2
2,NM_017436:c.548T>A,0.51,0.48,1,1.398952,1.595533,0.181255,NaN,BAG2,BAG2
3,NM_017436:c.656C>T,0.51,0.75,1,1.398952,4.833744,3.167002,yes,BAG2,BAG2
4,NM_015665:c.43C>A,0.20,0.35,0,6.765101,9.823864,2.820292,NaN,BAG2,BAG2


In [52]:
df['elisa_score_diff'] = df['mut_elisa_score'] - df['wt_elisa_score']
df['interaction_score_diff'] = df['mut_interaction_score'] - df['wt_interaction_score']

In [53]:
df.head()

,mutation_refseq_nt,wt_elisa_score,mut_elisa_score,both_expressed,wt_interaction_score,mut_interaction_score,differential_z_score,interact,quality_control_factor_common,quality_control_factor_official,elisa_score_diff,interaction_score_diff
0,NM_017436:c.287G>A,0.51,0.79,1,1.398952,4.766989,3.105452,yes,BAG2,BAG2,0.28,3.368036
1,NM_017436:c.299C>T,0.51,0.71,1,1.398952,5.029331,3.347341,yes,BAG2,BAG2,0.20,3.630378
2,NM_017436:c.548T>A,0.51,0.48,1,1.398952,1.595533,0.181255,NaN,BAG2,BAG2,-0.03,0.196581
3,NM_017436:c.656C>T,0.51,0.75,1,1.398952,4.833744,3.167002,yes,BAG2,BAG2,0.24,3.434791
4,NM_015665:c.43C>A,0.20,0.35,0,6.765101,9.823864,2.820292,NaN,BAG2,BAG2,0.15,3.058764


In [54]:
db.import_table(
    df, 
    'taipale_core_histone', [
        [('mutation_refseq_nt', ), False],
    ],
)

### mmc2_s2b

In [55]:
df = mmc2_s2b.copy()

In [56]:
df.columns = [datapkg.format_column(c) for c in df.columns]

In [57]:
df.head()

,mutation_refseq_nt,foldx_value
0,NM_000019:c.218A>C,7.841320
1,NM_000019:c.278A>G,0.236651
2,NM_000019:c.380C>T,0.215789
3,NM_000019:c.395C>G,1.050880
4,NM_005891:c.761T>C,2.247490


In [ ]:
df.head()

In [58]:
db.import_table(
    df, 
    'taipale_core_foldx', [
        [('mutation_refseq_nt', ), False],
    ],
)